In [1]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers

#Import the helper functions
#from helperFunctions import CSP
#from helperFunctions import GetCombinedData_HaLT as GetHALT
#from helperFunctions import GetCombinedData_CLA as GetCLA
#from helperFunctions import GetCombinedData_5F as Get5F
#from helperFunctions import GetCombinedData_FreeForm as GetFree

from helperFunctions import GetCombinedData_CLA_WBSS as GetCLA
from helperFunctions import GetCombinedData_HaLT_WBSS as GetHALT
from helperFunctions import GetCombinedData_5F as Get5F
from helperFunctions import GetCombinedData_FreeForm_WBSS as GetFree


print("Available paradigms:\nHand, Leg, Tongue (HALT)\nClassic - L/R Hand (CLA)\nFingers (5F)")
paradigm = input('Enter desired paradigm: ');
directoryPath = input('Enter the relative path to data: ');
if(paradigm == 'HALT'):
    Data, Targets, DataCSP, TargetsCSP, DataICA, TargetsICA = GetHALT(directoryPath, True);
    numOutputs = 5;
elif(paradigm == 'CLA'):
    Data, Targets, DataCSP, TargetsCSP, DataICA, TargetsICA = GetCLA(directoryPath, True);
    print("############")
    print("Processing FreeForm data for further testing")
    FreeData, FreeTargets, FreeDataCSP, FreeTargetsCSP, FreeDataICA, FreeTargetsICA = GetFree('../../../matDown/FREEFORM_DATA/', True);
    numOutputs = 2;
elif(paradigm == '5F'):
    #Data, Targets, DataCSP, TargetsCSP, DataICA, TargetsICA = Get5F(directoryPath, True);
    Data, Targets, DataCSP, TargetsCSP = Get5F(directoryPath, True);
    numOutputs = 5;
else:
    print("Error: Invalid paradigm {}".format(paradigm));
    quit()

enableDropout = True;
dropoutPercentage = 0.3;

#Run GetCombinedData to pull the datasets from multiple subjects into a single set
#Data, Targets, DataCSP, TargetsCSP = GetHALT('../../../matDown/HaLT_Data', True);

## Split into train and test sets
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(Data, Targets, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

Available paradigms:
Hand, Leg, Tongue (HALT)
Classic - L/R Hand (CLA)
Fingers (5F)
Enter desired paradigm: 5F
Enter the relative path to data: ../../../matDown/5F_Data/
Processing dataset 1 of 10
Processing dataset 2 of 10
Processing dataset 3 of 10
Processing dataset 4 of 10
Processing dataset 5 of 10
Processing dataset 6 of 10
Processing dataset 7 of 10
Processing dataset 8 of 10
Processing dataset 9 of 10
Processing dataset 10 of 10


/Users/daniel/Documents/GitHub/ECE228ECGProject/JupyterNotebooks/helperFunctions.py:87: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


## Building  and training the model
After processing the data, we build the LSTM model.
We use 100 units in the first LSTM layer, 50 units in the second LSTM layer, and 2 or 5 dense units in the final layer (depending on the paradigm).

We first train the network on non-CSP data.

In [2]:
LSTM_EEG = Sequential()
LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(LSTM((50), return_sequences=False))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(Dense((numOutputs),activation='sigmoid'))

LSTM_EEG.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           550000    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 255       
Total params: 580,455
Trainable params: 580,455
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
372/372 - 11s - loss: 0.5111 - accuracy: 0.2149
Epoch 2/30
372/372 - 10s - loss: 0.4969 - accuracy: 0.250

## Evaluating the model
Now, we use the model to predict the outputs for the test set, and compare them to the target outputs.

In [3]:
predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

#################################
#################################
Training Performance:
Correct MI Prediction: 4317
Incorrect MI Prediction: 1630
Percent Accuracy: 72.591%
#################################
#################################
Testing Performance:
Correct MI Prediction: 1480
Incorrect MI Prediction: 1069
Percent Accuracy: 58.062%
#################################
#################################


## Using CSP Data
Now, we train the model using the CSP data and then evaluate it

In [4]:
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(DataCSP, TargetsCSP, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

Epoch 1/30
372/372 - 11s - loss: 0.4942 - accuracy: 0.3027
Epoch 2/30
372/372 - 11s - loss: 0.4665 - accuracy: 0.3728
Epoch 3/30
372/372 - 10s - loss: 0.4530 - accuracy: 0.4024
Epoch 4/30
372/372 - 10s - loss: 0.4455 - accuracy: 0.4221
Epoch 5/30
372/372 - 10s - loss: 0.4306 - accuracy: 0.4606
Epoch 6/30
372/372 - 10s - loss: 0.4180 - accuracy: 0.4843
Epoch 7/30
372/372 - 10s - loss: 0.4122 - accuracy: 0.4922
Epoch 8/30
372/372 - 10s - loss: 0.3954 - accuracy: 0.5189
Epoch 9/30
372/372 - 10s - loss: 0.3806 - accuracy: 0.5409
Epoch 10/30
372/372 - 11s - loss: 0.3728 - accuracy: 0.5581
Epoch 11/30
372/372 - 11s - loss: 0.3611 - accuracy: 0.5729
Epoch 12/30
372/372 - 11s - loss: 0.3496 - accuracy: 0.5971
Epoch 13/30
372/372 - 11s - loss: 0.3387 - accuracy: 0.6121
Epoch 14/30
372/372 - 11s - loss: 0.3320 - accuracy: 0.6090
Epoch 15/30
372/372 - 11s - loss: 0.3211 - accuracy: 0.6284
Epoch 16/30
372/372 - 10s - loss: 0.3114 - accuracy: 0.6479
Epoch 17/30
372/372 - 11s - loss: 0.3068 - accura

## Using Wavelet-BSS Algorithm
In the following cell, we train the network on the Wavelet-BSS pre-processed data to compare to the previous approaches.

In [5]:
## Split into train and test sets

DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(DataICA, TargetsICA, test_size=0.3, random_state=0)

#NumSamples = number of rows
numSamples = DataTrain.shape[0];
if(len(DataTrain.shape) < 3):
    numTime = DataTrain.shape[1];
    numInputs = 1;
else:
    numTime = DataTrain.shape[1];
    numInputs = DataTrain.shape[2];

numSamplesTest = DataTest.shape[0];
if(len(DataTest.shape) < 3):
    numTimeTest = DataTest.shape[1];
    numInputsTest = 1;
else:
    numTimeTest = DataTest.shape[1];
    numInputsTest = DataTest.shape[2];
    
## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((numSamples, numTime, numInputs))
DataTestRe = DataTest.reshape((numSamplesTest, numTimeTest, numInputsTest))

LSTM_EEG_ICA = Sequential()
LSTM_EEG_ICA.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
if(enableDropout):
    LSTM_EEG_ICA.add(Dropout(dropoutPercentage))
LSTM_EEG_ICA.add(LSTM((50), return_sequences=False))
if(enableDropout):
    LSTM_EEG_ICA.add(Dropout(dropoutPercentage))
LSTM_EEG_ICA.add(Dense((numOutputs),activation='sigmoid'))

LSTM_EEG_ICA.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG_ICA.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = LSTM_EEG_ICA.fit(DataTrainRe, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG_ICA.predict(DataTestRe)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG_ICA.predict(DataTrainRe)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_ICA = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_ICA = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_ICA))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_ICA))
print("#################################")
print("#################################")

NameError: name 'DataICA' is not defined

## Optional testing of small FreeForm Datasets
The FREEFORM datasets are a collection of datasets (3 of them) that are self-paced. There may not be enough data to train the network on just the FreeForm dataset. If you do train it on that dataset, you might have issues with overfitting. So, if the paradigm was CLA, which is similar to FreeForm, we can evaluate the model on FreeForm as well.

In [ ]:
if(paradigm == 'CLA'):
    ## Have to check and make sure that the FreeData is the same dimensions as the CLA data
    lenFree = FreeDataCSP.shape[2];
    if(lenFree < len(DataCSP)):
        FreeDataCSP = FreeDataCSP[:, :, 0:DataCSP.shape[2]];
        
    predictionsFree = LSTM_EEG.predict(FreeDataCSP)
    predictionsFree[predictionsFree>0.5] = 1;
    predictionsFree[predictionsFree<=0.5] = 0;
    comparisonArray = predictionsFree == FreeTargetsCSP;

    correctCount = 0
    for boolValues in comparisonArray:
        if(boolValues[0] & boolValues[1]):
            correctCount += 1
    falseCount = FreeDataCSP.shape[0] - correctCount


    # In[87]:
    Accuracy = (correctCount*100/FreeDataCSP.shape[0]);

    print("#################################")
    print("#################################")
    print("FreeForm Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCount, falseCount, Accuracy))
    print("#################################")
    print("#################################")

In [ ]:
DataTrainRe.shape